# 5. Magic Methods

 So far, we have seen many special methods with fixed names, such as __init__, __str__, etc. . We also call them dunder methods, because they all the form __<name>__.

So what's magic about them? The answer is, you don't have to invoke them directly. The invocation is realized behind the scenes. For example, when you create an instance x of a class A with the statement "x = A()", Python will do the necessary calls to __new__ and __init__.


# 5.1 A simple example

We had used the plus(+) sign to add numerical values, to concatenate strings or to combine lists. In fact to use the plus on two object, you need to implement the __add__ dunder function.

In below example, I decide that dog1+dog2 will concatenate their name

In [2]:
class Dog:
    def __init__(self,name):
        self.name=name
    def __add__(self, other):
        return Dog(self.name+other.name)

In [3]:
d1=Dog("Toto")
d2=Dog("Titi")
d3=d1+d2

print(d3.name)

TotoTiti


So you can see, we don't need to call __add__, it's been called automatically when we use +.

## 5.2 Overview of Magic Methods

### 5.2.1 Binary Operators
```text
Operator	Method
+	object.__add__(self, other)
-	object.__sub__(self, other)
*	object.__mul__(self, other)
//	object.__floordiv__(self, other)
/	object.__truediv__(self, other)
%	object.__mod__(self, other)
**	object.__pow__(self, other[, modulo])
<<	object.__lshift__(self, other)
>>	object.__rshift__(self, other)
&	object.__and__(self, other)
^	object.__xor__(self, other)
|	object.__or__(self, other)
```

### 5.2.2Extended Assignments

```text
Operator	Method
+=	object.__iadd__(self, other)
-=	object.__isub__(self, other)
*=	object.__imul__(self, other)
/=	object.__idiv__(self, other)
//=	object.__ifloordiv__(self, other)
%=	object.__imod__(self, other)
**=	object.__ipow__(self, other[, modulo])
<<=	object.__ilshift__(self, other)
>>=	object.__irshift__(self, other)
&=	object.__iand__(self, other)
^=	object.__ixor__(self, other)
|=	object.__ior__(self, other)
```

### 5.2.3 Unary Operators
```text
Operator	Method
-	object.__neg__(self)
+	object.__pos__(self)
abs()	object.__abs__(self)
~	object.__invert__(self)
complex()	object.__complex__(self)
int()	object.__int__(self)
long()	object.__long__(self)
float()	object.__float__(self)
oct()	object.__oct__(self)
hex()	object.__hex__(self)
```

### 5.2.4 Comparison Operators
```text
Operator	Method
<	object.__lt__(self, other)
<=	object.__le__(self, other)
==	object.__eq__(self, other)
!=	object.__ne__(self, other)
>=	object.__ge__(self, other)
>	object.__gt__(self, other)
```

## 5.3 A more complex example

In [6]:
class Length:
    __metric = {"mm" : 0.001, "cm" : 0.01, "m" : 1, "km" : 1000,
                "in" : 0.0254, "ft" : 0.3048, "yd" : 0.9144,
                "mi" : 1609.344 }
    def __init__(self, value, unit = "m" ):
        self.value = value
        self.unit = unit
    def Converse2Metres(self):
        return self.value * Length.__metric[self.unit]
    def __add__(self, other):
        l = self.Converse2Metres() + other.Converse2Metres()
        return Length(l / Length.__metric[self.unit], self.unit )
    # print for end user
    def __str__(self):
        return f"{str(self.Converse2Metres())} meters"
    # print for developer to debug
    def __repr__(self):
        return "Length(" + str(self.value) + ", '" + self.unit + "')"

In [9]:
l1=Length(2.56)
print(l1)
print(repr(l1))

l2=Length(3,"yd")
print(l2)
print(repr(l2))


print( l1+ l2 + Length(7.8,"in") + Length(7.03,"cm"))


2.56 meters
Length(2.56, 'm')
2.7432 meters
Length(3, 'yd')
5.57162 meters


## 5.4 Some important dunder function

Some dunder function are more important, because they can change the behavior of the class dramatically. Below are some examples.

### 5.4.1 __iter__ and __next__

In chapiter 3.11 Iterator_VS_Iterable, we have explained that if a class implement __iter__ which returns an iterator, then it's an **iterable class**.
If a class implements both __iter__(return self) and __next__, then this class is an **iterator class**

Below Fib function is an iterator class.

In [11]:
class Fib(object):
    def __init__(self):
        self.a, self.b = 0, 1

    def __iter__(self):
        return self

    def __next__(self):
        self.a, self.b = self.b, self.a + self.b # calculate next value
        if self.a > 100000: # loop exit condition
            raise StopIteration()
        return self.a

In [12]:
for n in Fib():
    print(n)

1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
1597
2584
4181
6765
10946
17711
28657
46368
75025


## 5.4.2 __getitem__

If you want a function that can get item by its index, you need to implement __getitem__.


In [15]:
class Fib(object):
    def __getitem__(self, n):
        if isinstance(n, int): # n is an index
            a, b = 1, 1
            for x in range(n):
                a, b = b, a + b
            return a
        if isinstance(n, slice): # n is a slice
            start = n.start
            stop = n.stop
            if start is None:
                start = 0
            a, b = 1, 1
            L = []
            for x in range(stop):
                if x >= start:
                    L.append(a)
                a, b = b, a + b
            return L

In [16]:
f = Fib()
print(f[10])
print(f[15])

89
987


In [17]:
f[0:5]

[1, 1, 2, 3, 5]

## 5.4.3 __getattr__  and __getattribute__

__getattr__ is called when an attribute lookup has not found the attribute in the usual places (i.e. it is not an instance attribute nor is it found in the class tree for self).


 Check below example, if we call name, it's ok, If we call score, it will raise exception, because it does not exist in Student.

In [18]:
class Student(object):

    def __init__(self):
        self.name = 'Michael'

In [19]:
s=Student()
s.name

'Michael'

In [20]:
s.score

AttributeError: 'Student' object has no attribute 'score'

If we add __getattr__, then we can treat all call of attributes dynamically.

In [21]:
class Student(object):

    def __init__(self):
        self.name = 'Michael'

    def __getattr__(self, item):
        if item=="score":
            return 100
        elif item=="sex":
            return "Masculin"
        else:
            raise AttributeError('\'Student\' object has no attribute \'%s\'' % item)

In [22]:
s=Student()
print(s.name)
print(s.score)
print(s.sex)

Michael
100
Masculin


In [23]:
print(s.toto)

AttributeError: 'Student' object has no attribute 'toto'

### A more useful example for __getattr__

Now, many site provide REST API：

http://api.server/user/friends
http://api.server/user/timeline/list

If you want to write a SDK，for each URL you need to have an endpoint in your API，and if the url change, you need to change the SDK too.

With __getattr__，we can write a chain that builds the url dynamically：

In [24]:
class Chain(object):

    def __init__(self, path=''):
        self._path = path

    def __getattr__(self, path):
        return Chain('%s/%s' % (self._path, path))

    def __str__(self):
        return self._path


    __repr__ = __str__

In [25]:
Chain().status.user.timeline.list

/status/user/timeline/list

For the api that need to pass parameters:
GET /users/:user/repos
We can use below function.

In [26]:
Chain().users('michael').repos

TypeError: 'Chain' object is not callable

It says it's not callable, we will see how to make an object callable below.

### __getattribute__

Unlike __getattr__ will be used only if nothing can be used, __getattribute__ will be used all the time, even thought there are normal instance attribute found. So be careful, when you use it. It can quickly turn your class into a mess. Check below example.

In [28]:
class Yeah(object):
    def __init__(self, name):
        self.name = name
    # Gets called when an attribute is accessed
    def __getattribute__(self, item):
        print ('__getattribute__ ', item)
        # Calling the super class to avoid recursion
        return super(Yeah, self).__getattribute__(item)

    # Gets called when the item is not found via __getattribute__
    def __getattr__(self, item):
        print ('__getattr__ ', item)
        return super(Yeah, self).__setattr__(item, 'orphan')

In [29]:
y1 = Yeah('yes')

In [30]:
y1.name

__getattribute__  name


'yes'

In [31]:
y1.foo

__getattribute__  foo
__getattr__  foo


In [32]:
y1.foo

__getattribute__  foo


'orphan'

In [33]:
y1.__dict__

__getattribute__  __dict__


{'name': 'yes', 'shape': 'orphan', 'foo': 'orphan'}